In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install lxml

     |████████████████████████████████| 5.5MB 6.2MB/s eta 0:00:01�█████████▎     | 4.5MB 6.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


Part 1 - Data extraction from wikipedia

In [4]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [5]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#Drop a row if it contains a certain value
df1= df[df['Borough'] != 'Not assigned']
df1.head().reset_index(drop=True)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
#Since borough column does not have numerical value it will not change due to use of sum
Tdf = df1.groupby('Postal Code').agg(Borough=('Borough','sum'),Neighborhood=('Neighborhood', ', '.join)).reset_index()

In [8]:
Tdf.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
Tdf['Neighborhood'].replace("Not assigned", Tdf["Borough"],inplace=True)
Tdf.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
Tdf.shape

(103, 3)

PART-2 Data frame with location data

In [11]:
ll=pd.read_csv('Geospatial_Coordinates.csv')
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
ll.shape

(103, 3)

In [13]:
lldf=pd.merge(Tdf,ll,on=['Postal Code'], how='inner')

In [14]:
lldf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
lldf['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

Part 3 - Visualizing and analyzing the Toronto neighborhood

In [16]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(lldf['Latitude'], lldf['Longitude'], lldf['Borough'], lldf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [19]:
york = lldf[lldf['Borough']=='York'].reset_index(drop=True)
york

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
2,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
3,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [20]:
#york latitude and longitude
ylat=43.6896191 
ylong=-79.479188

In [22]:
# create map of etobicoke using latitude and longitude values
map_etob = folium.Map(location=[ylat, ylong], zoom_start=10)

# add markers to map
for lat, lng, label in zip(york['Latitude'], york['Longitude'], york['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etob)  
    
map_etob

In [23]:
CLIENT_ID = 'BLX0GALF5NO0MUY0X0BKRFRLOWH1TJQM45WLTV3OL4MZ25TR' # your Foursquare ID
CLIENT_SECRET = 'ZURXSS2P3O5SRCVBRXGE1HRZXEN5XH54Q4NN5KZRGFD345BE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: '+ CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BLX0GALF5NO0MUY0X0BKRFRLOWH1TJQM45WLTV3OL4MZ25TR
CLIENT_SECRET:ZURXSS2P3O5SRCVBRXGE1HRZXEN5XH54Q4NN5KZRGFD345BE


In [24]:
neighbourhood_latitude = york.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = york.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = york.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Humewood-Cedarvale are 43.6937813, -79.42819140000002.


Exploring top 100 locations of  Caledonia-Fairbanks

In [25]:
n_lat=43.69
n_lon=-79.43

In [26]:
radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, n_lat, n_lon, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=BLX0GALF5NO0MUY0X0BKRFRLOWH1TJQM45WLTV3OL4MZ25TR&client_secret=ZURXSS2P3O5SRCVBRXGE1HRZXEN5XH54Q4NN5KZRGFD345BE&ll=43.69,-79.43&v=20180605&radius=1000&limit=100'

In [27]:
york_results = requests.get(url).json()


In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
venues = york_results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Cedarvale Park,Field,43.692535,-79.428705
1,Cedarvale Ravine,Trail,43.690188,-79.426106
2,Pukka Restaurant,Indian Restaurant,43.681055,-79.429187
3,Bar Ape,Ice Cream Shop,43.681570,-79.427797
4,CocoaLatte,Café,43.681768,-79.425158


In [30]:
nearby_venues.shape

(38, 4)

In [94]:
ylat=43.6896191 
ylong=-79.479188

Explore Neighborhoods in York

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:


york_venues = getNearbyVenues(names=york['Neighborhood'],
                                   latitudes=york['Latitude'],
                                   longitudes=york['Longitude']
                                  )


Humewood-Cedarvale
Caledonia-Fairbanks
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Runnymede, The Junction North
Weston


In [33]:
print(york_venues.shape)
york_venues

(19, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Humewood-Cedarvale,43.693781,-79.428191,Cedarvale Park,43.692535,-79.428705,Field
1,Humewood-Cedarvale,43.693781,-79.428191,Cedarvale Ravine,43.690188,-79.426106,Trail
2,Humewood-Cedarvale,43.693781,-79.428191,Cedarvale Tennis Courts,43.692744,-79.432244,Tennis Court
3,Humewood-Cedarvale,43.693781,-79.428191,Phil White Arena,43.691303,-79.431761,Hockey Arena
4,Caledonia-Fairbanks,43.689026,-79.453512,Nairn Park,43.690654,-79.456300,Park
5,Caledonia-Fairbanks,43.689026,-79.453512,Maximum Woman,43.690651,-79.456333,Women's Store
6,Caledonia-Fairbanks,43.689026,-79.453512,Jiang,43.690985,-79.450103,Convenience Store
7,Caledonia-Fairbanks,43.689026,-79.453512,Landmark Restaurant,43.690400,-79.447702,Bar
8,Caledonia-Fairbanks,43.689026,-79.453512,Fairbank Memorial Park,43.692028,-79.448924,Park
9,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,Subway,43.690218,-79.474050,Sandwich Place


In [34]:
york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Caledonia-Fairbanks,5,5,5,5,5,5
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",4,4,4,4,4,4
Humewood-Cedarvale,4,4,4,4,4,4
"Runnymede, The Junction North",3,3,3,3,3,3
Weston,3,3,3,3,3,3


In [35]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 13 uniques categories.


In [36]:
# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighborhood'] = york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head()

,Neighborhood,Bar,Bus Line,Convenience Store,Discount Store,Field,Hockey Arena,Park,Pizza Place,Sandwich Place,Skating Rink,Tennis Court,Trail,Women's Store
0,Humewood-Cedarvale,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Humewood-Cedarvale,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Humewood-Cedarvale,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Humewood-Cedarvale,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Caledonia-Fairbanks,0,0,0,0,0,0,1,0,0,0,0,0,0


In [37]:
york_grouped = york_onehot.groupby('Neighborhood').mean().reset_index()
york_grouped

,Neighborhood,Bar,Bus Line,Convenience Store,Discount Store,Field,Hockey Arena,Park,Pizza Place,Sandwich Place,Skating Rink,Tennis Court,Trail,Women's Store
0,Caledonia-Fairbanks,0.20,0.000000,0.200000,0.00,0.00,0.00,0.400000,0.000000,0.00,0.00,0.00,0.00,0.2
1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.25,0.000000,0.000000,0.25,0.00,0.00,0.000000,0.000000,0.25,0.25,0.00,0.00,0.0
2,Humewood-Cedarvale,0.00,0.000000,0.000000,0.00,0.25,0.25,0.000000,0.000000,0.00,0.00,0.25,0.25,0.0
3,"Runnymede, The Junction North",0.00,0.333333,0.333333,0.00,0.00,0.00,0.000000,0.333333,0.00,0.00,0.00,0.00,0.0
4,Weston,0.00,0.000000,0.333333,0.00,0.00,0.00,0.666667,0.000000,0.00,0.00,0.00,0.00,0.0


In [38]:
num_top_venues = 5

for hood in york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = york_grouped[york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caledonia-Fairbanks----
               venue  freq
0               Park   0.4
1                Bar   0.2
2  Convenience Store   0.2
3      Women's Store   0.2
4           Bus Line   0.0


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
            venue  freq
0             Bar  0.25
1  Discount Store  0.25
2  Sandwich Place  0.25
3    Skating Rink  0.25
4        Bus Line  0.00


----Humewood-Cedarvale----
          venue  freq
0         Field  0.25
1  Hockey Arena  0.25
2  Tennis Court  0.25
3         Trail  0.25
4           Bar  0.00


----Runnymede, The Junction North----
               venue  freq
0           Bus Line  0.33
1  Convenience Store  0.33
2        Pizza Place  0.33
3                Bar  0.00
4     Discount Store  0.00


----Weston----
               venue  freq
0               Park  0.67
1  Convenience Store  0.33
2                Bar  0.00
3           Bus Line  0.00
4     Discount Store  0.00




In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = york_grouped['Neighborhood']

for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Caledonia-Fairbanks,Park,Women's Store,Convenience Store,Bar,Trail,Tennis Court,Skating Rink,Sandwich Place,Pizza Place,Hockey Arena
1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Skating Rink,Sandwich Place,Discount Store,Bar,Women's Store,Trail,Tennis Court,Pizza Place,Park,Hockey Arena
2,Humewood-Cedarvale,Trail,Tennis Court,Hockey Arena,Field,Women's Store,Skating Rink,Sandwich Place,Pizza Place,Park,Discount Store
3,"Runnymede, The Junction North",Pizza Place,Convenience Store,Bus Line,Women's Store,Trail,Tennis Court,Skating Rink,Sandwich Place,Park,Hockey Arena
4,Weston,Park,Convenience Store,Women's Store,Trail,Tennis Court,Skating Rink,Sandwich Place,Pizza Place,Hockey Arena,Field


Clustering

In [42]:
# set number of clusters
kclusters = 3

york_grouped_clustering = york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 2, 0], dtype=int32)

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_merged = york

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

york_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,1,Trail,Tennis Court,Hockey Arena,Field,Women's Store,Skating Rink,Sandwich Place,Pizza Place,Park,Discount Store
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0,Park,Women's Store,Convenience Store,Bar,Trail,Tennis Court,Skating Rink,Sandwich Place,Pizza Place,Hockey Arena
2,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,1,Skating Rink,Sandwich Place,Discount Store,Bar,Women's Store,Trail,Tennis Court,Pizza Place,Park,Hockey Arena
3,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262,2,Pizza Place,Convenience Store,Bus Line,Women's Store,Trail,Tennis Court,Skating Rink,Sandwich Place,Park,Hockey Arena
4,M9N,York,Weston,43.706876,-79.518188,0,Park,Convenience Store,Women's Store,Trail,Tennis Court,Skating Rink,Sandwich Place,Pizza Place,Hockey Arena,Field


In [45]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighborhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,York,0,Park,Women's Store,Convenience Store,Bar,Trail,Tennis Court,Skating Rink,Sandwich Place,Pizza Place,Hockey Arena
4,York,0,Park,Convenience Store,Women's Store,Trail,Tennis Court,Skating Rink,Sandwich Place,Pizza Place,Hockey Arena,Field
